# Eat Safe, Love

## Notebook Set Up

In [41]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [42]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [43]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']

In [44]:
# review the collections in our database
print(uk_food_db.list_collection_names())

['establishments']


In [45]:
# assign the collection to a variable
establishments = uk_food_db['establishments']

### Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [46]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
bad_score_count = establishments.count_documents(query)

# limit the results to the first 2
limit = 2
result_two = list(establishments.find(query).limit(limit))
result_full = list(establishments.find(query))

# Display the first document in the results using pprint
pprint(bad_score_count)
pprint(list(result_two))

0
[]


In [47]:
# Convert the result to a Pandas DataFrame
bad_hygiene_df = pd.DataFrame(result_full)
bad_hygiene_df

# Display the number of rows in the DataFrame
print(len(bad_hygiene_df))

# Display the first 10 rows of the DataFrame
result = bad_hygiene_df.head(10)
result

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [48]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
rating_query = {"RatingValue": {'$in': ['4', '5']}, 'LocalAuthorityName': { '$regex': "London" } }

# Use count_documents to display the number of documents in the result
results = list(establishments.find(rating_query))
count = establishments.count_documents(rating_query)

# Display the first document in the results using pprint
pprint(count)
pprint(list(results))

0
[]


In [49]:
# Convert the result to a Pandas DataFrame
four_score_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(len(four_score_df))

# Display the first 10 rows of the DataFrame
results = four_score_df.head(10)
results

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [50]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
query_business = {'BusinessName': 'Penang Flavours'}
results = list(establishments.find(query_business))
for result in results:
    longitude = float(result['geocode']['longitude'].to_decimal())
    latitude = float(result['geocode']['latitude'].to_decimal())
degree_search = 0.01
query = list(establishments.find({'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
                                  'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
                                  "RatingValue": '5'}))
 

# Print the results
pprint(query)

[]


In [51]:
# Convert result to Pandas DataFrame
five_score_df = pd.DataFrame(query)
results = five_score_df.head(10)
len(five_score_df)

0

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [52]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match': {'scores.Hygiene': 0 }}

group_query = {'$group': {'_id': "$LocalAuthorityName", 
                          'count': { '$sum': 1 }}}

sort_values = {'$sort': { 'count': -1 }}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
pprint(len(results))
# Print the first 10 results
pprint(results[0:10])

0
[]


In [53]:
# Convert the result to a Pandas DataFrame
zero_score_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(len(zero_score_df))

# Display the first 10 rows of the DataFrame
results = zero_score_df.head(10)
results

0


""
